In [1]:
from ipsl_dcpp.model.ipsl_dataset import IPSL_DCPP
import torch
import lightning as pl
from ipsl_dcpp.model.pangu import PanguWeather
from hydra import compose, initialize
from omegaconf import OmegaConf
import hydra
import os
work_dir = os.environ['WORK']
with initialize(version_base=None, config_path="../conf"):
    cfg = compose(config_name="config")


/gpfsdswork/projects/rech/mlr/udy16au/miniconda3/envs/env_gpu5/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
train_dataloader = torch.utils.data.DataLoader(IPSL_DCPP('train',cfg.experiment.lead_time_months,surface_variables=cfg.experiment.surface_variables,depth_variables=cfg.experiment.depth_variables),batch_size=1,shuffle=False,num_workers=1)
batch = next(iter(train_dataloader))
model = hydra.utils.instantiate(
    cfg.experiment.module,
    backbone=hydra.utils.instantiate(
        cfg.experiment.backbone,
    ),
    dataset=train_dataloader.dataset
)


/gpfsdswork/projects/rech/mlr/udy16au/miniconda3/envs/env_gpu5/lib/python3.11/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3526.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [3]:
# trainer = pl.Trainer(fast_dev_run=100)
trainer = pl.Trainer(fast_dev_run=2)
output = trainer.fit(model, train_dataloader)


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/gpfsdswork/projects/rech/mlr/udy16au/miniconda3/envs/env_gpu5/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_connector.py:67: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `lightning.pytorch` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
Running in `fast_dev_run` mode: will run the requested loop using 2 batch(es). Logging and checkpointing is suppressed.
/gpfsdswork/projects/rech/mlr/udy16au/miniconda3/envs/env_gpu5/lib/python3.11/site-packages/lightning/pytorch/trainer/configuration_validator.py:74: You defi

configure optimizers


SLURM auto-requeueing enabled. Setting signal handlers.
/gpfsdswork/projects/rech/mlr/udy16au/miniconda3/envs/env_gpu5/lib/python3.11/site-packages/lightning/pytorch/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Epoch 0:   0%|          | 0/2 [00:00<?, ?it/s] torch.Size([1, 1, 91, 143, 144])
torch.Size([1, 1, 3, 11, 143, 144])
torch.Size([1, 91, 143, 144])
torch.Size([1, 3, 11, 143, 144])
Epoch 0:  50%|█████     | 1/2 [00:13<00:13,  0.07it/s, train_loss=2.15e+24]torch.Size([1, 1, 91, 143, 144])
torch.Size([1, 1, 3, 11, 143, 144])
torch.Size([1, 91, 143, 144])
torch.Size([1, 3, 11, 143, 144])
Epoch 0: 100%|██████████| 2/2 [00:25<00:00,  0.08it/s, train_loss=2.37e+23]

`Trainer.fit` stopped: `max_steps=2` reached.


Epoch 0: 100%|██████████| 2/2 [00:25<00:00,  0.08it/s, train_loss=2.37e+23]


In [3]:
# batch = next(iter(train_dataloader))
model.forward(batch)


torch.Size([1, 1, 91, 143, 144])
torch.Size([1, 1, 3, 11, 143, 144])
torch.Size([1, 91, 143, 144])
torch.Size([1, 3, 11, 143, 144])


{'latent': tensor([[[ 0.8130, -0.0326, -0.6717,  ..., -0.5579, -0.1578, -0.2686],
          [ 0.5609, -0.0646, -0.6327,  ..., -0.3566, -0.3911, -0.2753],
          [ 0.4147, -0.1236, -0.5922,  ..., -0.3120, -0.4411, -0.2321],
          ...,
          [-0.0928,  0.1415,  0.0109,  ..., -0.4969,  0.0526, -0.0315],
          [-0.0928,  0.1415,  0.0109,  ..., -0.4969,  0.0526, -0.0315],
          [-0.0928,  0.1415,  0.0109,  ..., -0.4969,  0.0526, -0.0315]]],
        grad_fn=<AddBackward0>),
 'next_state_surface': tensor([[[[-0.1466,  0.3251, -0.1207,  ...,  0.2235,  0.1056,  0.1663],
           [-0.1220,  0.2658, -0.0548,  ...,  0.0443,  0.0195, -0.3379],
           [-0.3230,  0.2116, -0.1642,  ..., -0.0587, -0.5140, -0.0850],
           ...,
           [ 0.2026, -0.2701, -0.0720,  ..., -0.2524, -0.0793, -0.1097],
           [-0.1901, -0.2227,  0.1080,  ..., -0.2044,  0.1251, -0.2242],
           [ 0.2721, -0.1870,  0.2802,  ..., -0.1596,  0.3047,  0.0613]],
 
          [[ 0.2800,  0.2991,

In [9]:
import numpy as np 
np.load('../climatology_from_train.npy').shape

(117, 91)